In [8]:
import pandas as pd
import mysql.connector 
import sqlalchemy as sa

In [10]:
connection = mysql.connector.connect(host='localhost',
                                     user='root',
                                     password=password,
                                     db='learningsql')

query = "SELECT * FROM ACCOUNT"

In [12]:
pd.read_sql(query, connection).head(2)

,ACCOUNT_ID,AVAIL_BALANCE,CLOSE_DATE,LAST_ACTIVITY_DATE,OPEN_DATE,PENDING_BALANCE,STATUS,CUST_ID,OPEN_BRANCH_ID,OPEN_EMP_ID,PRODUCT_CD
0,1,1057.75,None,2005-01-04,2000-01-15,1057.75,ACTIVE,1,2,10,CHK
1,2,500.00,None,2004-12-19,2000-01-15,500.00,ACTIVE,1,2,10,SAV


In [13]:
sql = """SELECT a.ACCOUNT_ID, concat(i.LAST_NAME, ' ', i.FIRST_NAME) as FULL_NAME,
       c.ADDRESS, b.NAME AS BUSINESS_NAME, a.AVAIL_BALANCE FROM ACCOUNT a
    INNER JOIN CUSTOMER C on a.CUST_ID = C.CUST_ID
    LEFT JOIN INDIVIDUAL I on C.CUST_ID = I.CUST_ID
    LEFT JOIN BUSINESS B on C.CUST_ID = B.CUST_ID ORDER BY AVAIL_BALANCE desc LIMIT 3;"""

pd.read_sql(sql, connection)

,ACCOUNT_ID,FULL_NAME,ADDRESS,BUSINESS_NAME,AVAIL_BALANCE
0,29,None,4772 Presidential Way,AAA Insurance Inc.,50000.0
1,28,None,789 Main St,Superior Auto Body,38552.1
2,24,None,7 Industrial Way,Chilton Engineering,23575.1


In [15]:
department = "SELECT * FROM DEPARTMENT"
department = pd.read_sql(department, connection)

branch = "SELECT * FROM BRANCH"
branch = pd.read_sql(branch, connection)

employee = "SELECT * FROM employee"
employee = pd.read_sql(employee, connection)

account = "SELECT * FROM account"
account = pd.read_sql(account, connection)

# Tìm 3 employee quản lý 3 accounts có số dư cao nhất.

In [16]:
department.head(1)

,DEPT_ID,NAME
0,1,Operations


In [17]:
branch.head(1)

,BRANCH_ID,ADDRESS,CITY,NAME,STATE,ZIP_CODE
0,1,3882 Main St.,Waltham,Headquarters,MA,02451


In [18]:
employee.head(1)

,EMP_ID,END_DATE,FIRST_NAME,LAST_NAME,START_DATE,TITLE,ASSIGNED_BRANCH_ID,DEPT_ID,SUPERIOR_EMP_ID
0,1,None,Michael,Smith,2001-06-22,President,1,3,NaN


In [19]:
account.head(1)

,ACCOUNT_ID,AVAIL_BALANCE,CLOSE_DATE,LAST_ACTIVITY_DATE,OPEN_DATE,PENDING_BALANCE,STATUS,CUST_ID,OPEN_BRANCH_ID,OPEN_EMP_ID,PRODUCT_CD
0,1,1057.75,None,2005-01-04,2000-01-15,1057.75,ACTIVE,1,2,10,CHK


In [24]:
all_df = employee.merge(account, left_on='EMP_ID', how='inner', right_on='OPEN_EMP_ID') \
.merge(branch, how='inner', left_on='ASSIGNED_BRANCH_ID', right_on='BRANCH_ID') \
.merge(department, how='inner', left_on='DEPT_ID', right_on='DEPT_ID', suffixes=('_branch', '_deparment'))

In [25]:
all_df.columns

Index(['EMP_ID', 'END_DATE', 'FIRST_NAME', 'LAST_NAME', 'START_DATE', 'TITLE',
       'ASSIGNED_BRANCH_ID', 'DEPT_ID', 'SUPERIOR_EMP_ID', 'ACCOUNT_ID',
       'AVAIL_BALANCE', 'CLOSE_DATE', 'LAST_ACTIVITY_DATE', 'OPEN_DATE',
       'PENDING_BALANCE', 'STATUS', 'CUST_ID', 'OPEN_BRANCH_ID', 'OPEN_EMP_ID',
       'PRODUCT_CD', 'BRANCH_ID', 'ADDRESS', 'CITY', 'NAME_branch', 'STATE',
       'ZIP_CODE', 'NAME_deparment'],
      dtype='object')

In [29]:
all_df[['EMP_ID', 'FIRST_NAME', 'LAST_NAME', 'AVAIL_BALANCE', 
        'ADDRESS', 'CITY', 'NAME_branch', 'NAME_deparment']].nlargest(3, 'AVAIL_BALANCE', keep='all')

,EMP_ID,FIRST_NAME,LAST_NAME,AVAIL_BALANCE,ADDRESS,CITY,NAME_branch,NAME_deparment
17,13,John,Blake,50000.0,125 Presidential Way,Quincy,Quincy Branch,Operations
23,16,Theresa,Markham,38552.1,378 Maynard Ln.,Salem,So. NH Branch,Operations
21,16,Theresa,Markham,23575.1,378 Maynard Ln.,Salem,So. NH Branch,Operations
